# CNN Model - Previous Fire Data and Weather

In [1]:
# Load packages
import boto3
import io
import csv
import math
import numpy as np
import os
import pandas as pd
import pickle
import random

from datetime import datetime as dt
from matplotlib import pyplot as plt
from PIL import Image

### Variables and Hyperparameters

In [22]:
# s3 config
s3_client = boto3.client('s3')
bucket_name = 'hotzone'

# CNN config

# the desired height and width (in pixels) of the matrix to feed into the CNN
# 1 pixel side = 500 meters = 0.310686 miles
matrix_dim = 32


# test size for train/test split
test_size = 0.2

# training epochs
epoc = 10

## Pull Data from S3

In [6]:
def pull_data_from_s3(s3_client, bucket_name, key_name):
    '''
    Pulls pre-processed data from S3.

    Args:
        - s3_client: boto3 s3 client
        - bucket_name: name of bucket on s3 to pull data from
        - key_name: directory/file_name to pull data from
    Returns:
        - Nothing
    
    https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
    '''
    
    array_data = io.BytesIO()
    s3_client.download_fileobj(bucket_name, key_name, array_data)
    
    array_data.seek(0)
    array = pickle.load(array_data)

    return array

In [7]:
fire_key_name = 'test/fire_2016.pickle'
weather_key_name = 'test/weather_2016.pickle'
label_key_name = 'test/label_2016.pickle'

fire = pull_data_from_s3(s3_client, bucket_name, fire_key_name)
weather = pull_data_from_s3(s3_client, bucket_name, weather_key_name)
Y = pull_data_from_s3(s3_client, bucket_name, label_key_name)

## Build CNN

In [8]:
# import packages

from __future__ import print_function

import tensorflow as tf

import keras
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Conv1D, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, concatenate

Using TensorFlow backend.


In [9]:
# compute f1 score manually - taken from https://datascience.stackexchange.com/a/45166

def recall_m(y_true, y_pred):
    '''
    Computes recall.
    
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - recall: true positives / actual results
    '''
    
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_pos / (possible_pos + K.epsilon())

    return recall


def precision_m(y_true, y_pred):
    '''
    Computes precision.
    
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - precision: true positives / predicted results
    '''
    
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_pos / (predicted_pos + K.epsilon())
    
    return precision


def f1_score(y_true, y_pred):
    '''
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - score: f1 score
    '''
    
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return score

In [10]:
# Create model_2: image data, weather data, and fire speed/direction data with functional API

# Define image inputs shape
image_shape = fire[0].shape
image_inputs = Input(shape = image_shape)

# Define weather inputs shape
weather_shape = weather[0].shape
weather_inputs = Input(shape = weather_shape)

# Add layers for fire image interpretation
fire_1 = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid')(image_inputs)
fire_2 = Conv2D(32, kernel_size=(3, 3), activation='sigmoid')(fire_1)
fire_3 = Conv2D(64, kernel_size=(3, 3), activation='sigmoid')(fire_2)
fire_4 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(fire_3)
fire_5 = Dropout(0.2)(fire_4)
fire_6 = Flatten()(fire_5)
fire_7 = Dense(64, activation='sigmoid')(fire_6)

# Combine the layers
concat = concatenate([fire_7, weather_inputs])

# Final dense layer 
predictions = Dense(1, activation='sigmoid')(concat)

# Define the model
model_2 = Model(inputs=[image_inputs, weather_inputs], outputs=predictions)

In [11]:
%%time
# compile the model
model_2.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy', f1_score, tf.keras.metrics.AUC()]
)

CPU times: user 156 ms, sys: 0 ns, total: 156 ms
Wall time: 155 ms


In [14]:
%%time
# fit the model
model_2.fit(
    x = [fire, weather], 
    y = Y,
    validation_split = test_size, 
    epochs=epoc
)

Train on 5409 samples, validate on 1353 samples
Epoch 1/10
5409/5409 [==============================] - 7s 1ms/step - loss: 0.7074 - accuracy: 0.4981 - f1_score: 0.4063 - auc: 0.4695 - val_loss: 0.7007 - val_accuracy: 0.5137 - val_f1_score: 0.0000e+00 - val_auc: 0.4961
Epoch 2/10
5409/5409 [==============================] - 5s 910us/step - loss: 0.6035 - accuracy: 0.7029 - f1_score: 0.5884 - auc: 0.5515 - val_loss: 0.3852 - val_accuracy: 0.8322 - val_f1_score: 0.7824 - val_auc: 0.6581
Epoch 3/10
5409/5409 [==============================] - 5s 866us/step - loss: 0.2975 - accuracy: 0.8909 - f1_score: 0.8716 - auc: 0.7400 - val_loss: 0.2039 - val_accuracy: 0.9298 - val_f1_score: 0.9191 - val_auc: 0.8017
Epoch 4/10
5409/5409 [==============================] - 5s 926us/step - loss: 0.2030 - accuracy: 0.9338 - f1_score: 0.9296 - auc: 0.8407 - val_loss: 0.1553 - val_accuracy: 0.9342 - val_f1_score: 0.9243 - val_auc: 0.8686
Epoch 5/10
5409/5409 [==============================] - 5s 882us/step 

## Fire Prediction Functions

In [15]:
def show_images(images, cols = 1, titles = None):
    '''
    Display a list of images in a single figure with matplotlib. Taken from 
    https://gist.github.com/soply/f3eec2e79c165e39c9d540e916142ae1
    
    Args:
        - images: List of np.arrays compatible with plt.imshow.
    
        - cols (Default = 1): Number of columns in figure (number of rows is set to np.ceil(n_images/float(cols))).
    
        - titles: List of titles corresponding to each image. Must have the same length as titles.
    '''

    assert((titles is None)or (len(images) == len(titles)))
    
    n_images = len(images)
    
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    
    fig = plt.figure()
    
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        plt.imshow(image, cmap='inferno', interpolation='nearest')
        a.set_title(title)
    
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    
    plt.show()

In [16]:
def predict_day(fire_id, doy, model, matrix_dim):
    '''
    Predicts where fire will be on day "D + 1" for a provided fire_id and day of year, using a provided model.
    
    Args:
        - fire_id: ID of fire to be predicted
        - doy: Day of year to predict fire for
        - model: Model to use for predictions
        - matrix_dim: a hyperparameter for the height and width of the matrices fed into the CNN
    Returns:
        - prediction: numpy matrix of predicted fire probabilities
    '''

    side = int(matrix_dim/2)
    fire_id = str(fire_id)
    doy = str(doy)
    
    try:
        true_fire_map = fire_data_dict[fire_id][doy]
    except KeyError:
        print("Not a valid fire_id and day of year combination")

    weather_data_test = weather_data[doy]
    small_dataset_test = None

    for (day, (x, y)) in small_dataset:
        if day == doy:
            small_dataset_test = (day, (x, y))

    (day, (x, y)) = small_dataset_test
        
    x = np.pad(x, pad_width=matrix_dim, mode='constant', constant_values=0)
    y = np.pad(y, pad_width=matrix_dim, mode='constant', constant_values=0)
    
    vals = np.where((y == 1) | (y == 0))
    vals = list(zip(vals[0], vals[1]))
    
    values = []
    
    shape = x.shape
    prediction = np.zeros(shape)
    
    for (xi, yi) in vals:
        point = (xi, yi)
        
        xi_r = xi + side
        xi_l = xi - side
        yi_b = yi + side
        yi_t = yi - side

        m = x[xi_l:xi_r, yi_t:yi_b]
        
        weather = fetch_weather_data(max_values, normalized_weather, doy, xi, yi)

        y_label = y[xi, yi]
        
        if (weather is not None) and (m.shape == (matrix_dim, matrix_dim)):
            values.append((point, y_label, weather, m))
    
    
    for (point, y_label, w, f) in values:        
        fire = []
        weather = []
        Y = []

        fire.append(np.asarray(f))
        weather.append(np.asarray(w))
        Y.append(y_label)

        fire = np.asarray(fire)
        weather = np.asarray(weather)
        Y = np.asarray(Y)
        
        obs = len(fire)
        fire = fire.reshape(obs, matrix_dim, matrix_dim, 1)

        if model == model_2:
            val = model.predict([fire, weather])
        else:
            raise Exception
            
            
        prediction[point] = val
    
    x_start = matrix_dim
    x_end = 456 + matrix_dim
    y_start = matrix_dim
    y_end = 470 + matrix_dim
    
    prediction = prediction[y_start:y_end, x_start:x_end]
    
    return prediction

## Predict a Day of Fire

In [18]:
def getrasterdims(geotif_filename):
    
    """Get the top left and resolution for pixels in an array
    given the tif coordinate reference system
    Input: geotiff path+filename
    Output: tuple of the x,y coordinate of top left and resolutions of pixel

    Example to find out which pixel a given x,y coordinate refers to:
    pixel_column = (x-left)/xres
    pixel_row = (y-top)/yres
    """

    with rio.open(geotif_filename) as tif:
        profile = tif.profile

    left = profile['transform'][2]
    top = profile['transform'][5]

    xres = profile['transform'][0]
    yres = profile['transform'][4]

    return ((left,top),(xres,yres))

In [19]:
def getcoords(geotif,pixelrow,pixelcol):
    
    """Get the coordinates of a given pixel in the tif coordinate
    system
    Input: geotiff path+filename, and row,column of the pixel in question
    Output: tuple of the x,y coordinate in the geotif coordinate system
    """

    with rio.open(geotif) as tif:
        profile = tif.profile

    left = profile['transform'][2]
    top = profile['transform'][5]

    xres = profile['transform'][0]
    yres = profile['transform'][4]

    deltax = xres*pixelcol
    deltay = yres*pixelrow

    x = left+deltax
    y = top+deltay

    return (x,y)

In [20]:
def convert_polygon(fire_polygon):
    """Convert Coord ref system (CRS) from fire data (EPSG 3857) to map
    lat/long (EPSG 4326), polygon starts and ends on same point (to close it)

    Input: list containing tuples of x,y points in fire CRS that describe a polygon
    Output: list containing tuples of lat/long points that describe the polygon
    """

    # create polygon
    poly = geometry.Polygon([(p[0], p[1]) for p in fire_polygon])

    #CRS
    src_crs = "EPSG:3857"
    dst_crs = "EPSG:4326"

    # Create Geo DataFrame
    gfp = gpd.GeoDataFrame(index=[0],crs=src_crs,geometry=[poly])

    # Convert CRS
    gfp2 = gfp.to_crs(dst_crs)

    # pull data from dataframe
    polyout = gfp2.iloc[0]['geometry']

    # create list of tuples, but longitude is first
    l = list(map(tuple,np.asarray(polyout.exterior.coords)))
    l = list(map(lambda m: (m[0],m[1]), l))

    return l

In [23]:
%%time
# predict fire location or that day using model 2
pred = predict_day('140', '209', model_2, matrix_dim)

NameError: name 'fire_data_dict' is not defined